In [65]:
import pandas as pd

In [66]:
import geopandas as gpd

def carregar_geojson_em_dataframe(caminho_arquivo):
    try:
        # Carrega o GeoJSON usando o geopandas
        gdf = gpd.read_file(caminho_arquivo)
        return gdf
    except Exception as e:
        print(f"Erro ao abrir o arquivo GeoJSON: {e}")
        return None

acidentes_df = carregar_geojson_em_dataframe("../data/ANTT/BR-101-SC/acidentes_br101sc.geojson")
acidentes_df.head(2)

,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,mortos,concessionaria,data_hora,com_vitima,estado,ano,mes,latitude,longitude,geometry
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,0,viacosteira,2021-05-01 12:10:00,False,SC,2021,5,-28.425440,-48.857996,POINT (-48.85800 -28.42544)
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,0,viacosteira,2021-05-01 12:10:00,False,SC,2021,5,-28.426395,-48.859912,POINT (-48.85991 -28.42639)


In [67]:
# conta a quantidade de valores vazios em latitude
acidentes_df['latitude'].isnull().sum()

539

In [68]:
acidentes_df.columns

Index(['n_da_ocorrencia', 'km', 'trecho', 'sentido', 'tipo_de_acidente',
       'automovel', 'bicicleta', 'caminhao', 'moto', 'onibus', 'outros',
       'tracao_animal', 'transporte_de_cargas_especiais', 'trator_maquinas',
       'utilitarios', 'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos', 'concessionaria', 'data_hora',
       'com_vitima', 'estado', 'ano', 'mes', 'latitude', 'longitude',
       'geometry'],
      dtype='object')

In [69]:
estacoes_proximas = carregar_geojson_em_dataframe("estacoes_inmet.geojson")
estacoes_proximas

,regiao,estado,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,geometry
0,S,RS,MOSTARDAS,A878,-31.248333,-50.906389,3.82,2008-03-11,../data/INMET/2022/INMET_S_RS_A878_MOSTARDAS_0...,POINT (-50.90639 -31.24833)
1,CO,GO,ITUMBIARA,A035,-18.409722,-49.191944,491.17,2007-11-01,../data/INMET/2022/INMET_CO_GO_A035_ITUMBIARA_...,POINT (-49.19194 -18.40972)
2,S,RS,DOM PEDRITO,A881,-31.002500,-54.618056,150.00,2010-04-23,../data/INMET/2022/INMET_S_RS_A881_DOM PEDRITO...,POINT (-54.61806 -31.00250)
3,CO,MT,VILA BELA DA SANTISSIMA TRINDADE,A922,-15.062778,-59.873056,213.00,2006-12-01,../data/INMET/2022/INMET_CO_MT_A922_VILA BELA ...,POINT (-59.87306 -15.06278)
4,SE,MG,ALMENARA,A508,-16.166667,-40.687778,189.11,2002-12-15,../data/INMET/2022/INMET_SE_MG_A508_ALMENARA_0...,POINT (-40.68778 -16.16667)
...,...,...,...,...,...,...,...,...,...,...
2871,S,RS,CAMAQUA,A838,-30.807953,-51.834240,92.30,2006-12-12,../data/INMET/2021/INMET_S_RS_A838_CAMAQUA_01-...,POINT (-51.83424 -30.80795)
2872,NE,BA,BOM JESUS DA LAPA,A418,-13.251111,-43.405278,447.75,2007-05-18,../data/INMET/2021/INMET_NE_BA_A418_BOM JESUS ...,POINT (-43.40528 -13.25111)
2873,N,AM,S. G. DA CACHOEIRA,A134,-0.125207,-67.061246,79.67,2011-08-31,../data/INMET/2021/INMET_N_AM_A134_S. G. DA CA...,POINT (-67.06125 -0.12521)
2874,SE,MG,PASSOS,A516,-20.745237,-46.633916,781.70,2006-07-16,../data/INMET/2021/INMET_SE_MG_A516_PASSOS_01-...,POINT (-46.63392 -20.74524)


In [72]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

def find_nearest_weather_station(accidents_df, weather_stations_df, radius_km=50):
    # Copy the input DataFrames to avoid modifying the original DataFrames
    accidents_df = accidents_df.copy()
    weather_stations_df = weather_stations_df.copy()

    # Remover linhas com valores NaN em coordenadas
    accidents_df = accidents_df.dropna(subset=['latitude', 'longitude'])
    weather_stations_df = weather_stations_df.dropna(subset=['lat', 'long'])

    # Lista para armazenar resultados
    merged_data = []

    # Itera sobre cada linha do DataFrame de acidentes
    for _, accident_row in accidents_df.iterrows():

        accident_location = (accident_row['latitude'], accident_row['longitude'])

        print(f"Procurando estação climática mais próxima para o acidente em {accident_location}")

        # Calcula a distância para cada estação climática
        weather_stations_df['distance'] = weather_stations_df.apply(
            lambda row: geodesic(accident_location, (row['lat'], row['long'])).kilometers, axis=1
        )

        # Filtra estações dentro do raio especificado
        nearby_stations = weather_stations_df[weather_stations_df['distance'] <= radius_km]

        if not nearby_stations.empty:
            # Encontra a estação mais próxima
            nearest_station = nearby_stations.loc[nearby_stations['distance'].idxmin()]
            combined_row = {**accident_row.to_dict(), **nearest_station.to_dict()}
        else:
            # Se nenhuma estação estiver próxima, preencha com NaN
            combined_row = {**accident_row.to_dict(), **{col: np.nan for col in weather_stations_df.columns}}

        merged_data.append(combined_row)

    # Retorna o DataFrame combinado
    return pd.DataFrame(merged_data)

acidentes_sample = acidentes_df.sample(15)
# datas_climaticas = pd.read_csv("datas_climaticas.csv")

acidentes_com_estacoes = find_nearest_weather_station(acidentes_sample, estacoes_proximas)
acidentes_com_estacoes

Procurando estação climática mais próxima para o acidente em (-27.975972, -48.679295)
Procurando estação climática mais próxima para o acidente em (-26.60108, -48.72536)
Procurando estação climática mais próxima para o acidente em (-27.70439, -48.6586)
Procurando estação climática mais próxima para o acidente em (-26.85054, -48.72587)
Procurando estação climática mais próxima para o acidente em (-28.445025, -48.967059)
Procurando estação climática mais próxima para o acidente em (-26.85488, -48.72722)
Procurando estação climática mais próxima para o acidente em (-27.00178, -48.6376)
Procurando estação climática mais próxima para o acidente em (-27.47249, -48.64114)
Procurando estação climática mais próxima para o acidente em (-26.28057, -48.89053)
Procurando estação climática mais próxima para o acidente em (-26.60122, -48.72533)
Procurando estação climática mais próxima para o acidente em (-27.64085, -48.66786)
Procurando estação climática mais próxima para o acidente em (-26.32482, -

,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,geometry,regiao,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,distance
0,80,254.0,BR-101/SC,Decrescente,Tombamento,0,0,1,0,0,...,POINT (-48.620096 -27.60253),S,FLORIANOPOLIS,A806,-27.602530,-48.620096,4.87,2003-01-22,../data/INMET/2022/INMET_S_SC_A806_FLORIANOPOL...,41.792599
1,56,80.5,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,POINT (-48.76194444 -26.95083333),S,ITAJAI,A868,-26.950833,-48.761944,9.76,2010-06-24,../data/INMET/2022/INMET_S_SC_A868_ITAJAI_01-0...,38.923092
2,274,221.3,BR-101/SC,Decrescente,Colisão Lateral,0,0,1,1,0,...,POINT (-48.620096 -27.60253),S,FLORIANOPOLIS,A806,-27.602530,-48.620096,4.87,2003-01-22,../data/INMET/2022/INMET_S_SC_A806_FLORIANOPOL...,11.909783
3,280,112.7,BR-101/SC,Decrescente,Colisão Lateral,1,0,1,0,0,...,POINT (-48.76194444 -26.95083333),S,ITAJAI,A868,-26.950833,-48.761944,9.76,2010-06-24,../data/INMET/2022/INMET_S_SC_A868_ITAJAI_01-0...,11.676257
4,72,328.0,BR-101/SC,Crescente,Colisão - Traseira,1,0,0,1,0,...,POINT (-48.81333333 -28.60444444),S,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,34.36,2008-06-01,../data/INMET/2022/INMET_S_SC_A866_Laguna - F...,23.207126
5,390,113.2,BR-101/SC,Decrescente,Colisão Traseira,1,0,1,0,0,...,POINT (-48.76194444 -26.95083333),S,ITAJAI,A868,-26.950833,-48.761944,9.76,2010-06-24,../data/INMET/2022/INMET_S_SC_A868_ITAJAI_01-0...,11.177425
6,378,133.9,BR-101/SC,Crescente,Colisão Traseira,1,0,0,1,0,...,POINT (-48.76194444 -26.95083333),S,ITAJAI,A868,-26.950833,-48.761944,9.76,2010-06-24,../data/INMET/2022/INMET_S_SC_A868_ITAJAI_01-0...,13.573882
7,147,190.5,BR-101/SC,Decrescente,"Choque - Defensa, barreira ou submarino""""",1,0,0,0,0,...,POINT (-48.61999999 -27.6025),S,FLORIANOPOLIS,A806,-27.602500,-48.620000,4.87,2003-01-22,../data/INMET/2023/INMET_S_SC_A806_FLORIANOPOL...,14.557222
8,71,36.5,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,POINT (-48.641774 -26.081303),S,ITAPOA,A851,-26.081303,-48.641774,6.18,2007-06-06,../data/INMET/2020/INMET_S_SC_A851_ITAPOA_01-0...,33.252644
9,207,80.5,BR-101/SC,Crescente,Colisão Traseira,1,0,1,0,0,...,POINT (-48.76194444 -26.95083333),S,ITAJAI,A868,-26.950833,-48.761944,9.76,2010-06-24,../data/INMET/2022/INMET_S_SC_A868_ITAJAI_01-0...,38.907927


In [73]:
import pandas as pd
import os
import re

def load_and_concat_unique_data_with_metadata(df, years=[2020, 2021, 2022, 2023]):
    """
    Lê arquivos CSV baseados em valores únicos na coluna 'nome_arquivo' de um DataFrame,
    processa para anos especificados e concatena sem duplicações, adicionando as informações
    das 8 primeiras linhas como novas colunas.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna 'nome_arquivo' com caminhos dos arquivos.
        years (list): Lista de anos de interesse para filtrar os dados (default: [2020, 2021, 2022, 2023]).

    Retorna:
        pd.DataFrame: Um DataFrame concatenado sem duplicações, com as colunas extras.
    """
    # Obter valores únicos na coluna 'nome_arquivo'
    unique_files = df['nome_arquivo'].dropna().unique()

    # Inicializa o DataFrame vazio para concatenar os resultados
    concatenated_df = pd.DataFrame()

    # Expressão regular para encontrar o ano no nome do arquivo
    year_pattern = r'\d{4}'  # padrão para encontrar ano de 4 dígitos

    # Conjunto para armazenar arquivos processados e evitar duplicações
    processed_files = set()

    for base_path in unique_files:
        # Encontra o ano presente no nome do arquivo
        match = re.search(year_pattern, base_path)
        if match:
            current_year = match.group()  # Ano encontrado no caminho do arquivo
            
            for year in years:
                # Substitui o ano encontrado no caminho do arquivo pelo ano da iteração
                file_path = base_path.replace(current_year, str(year))

                if file_path not in processed_files and os.path.exists(file_path):
                    print(f"Lendo arquivo: {file_path}")
                    processed_files.add(file_path)  # Marca o arquivo como processado
                    
                    try:
                        # Lê as primeiras 8 linhas do arquivo para extrair as variáveis adicionais
                        with open(file_path, 'r', encoding='latin1') as file:
                            first_8_lines = [next(file) for _ in range(8)]

                        # Extraindo as variáveis a partir das 8 primeiras linhas
                        metadata = {
                            "REGIAO": first_8_lines[0].split(';')[1].strip(),
                            "UF": first_8_lines[1].split(';')[1].strip(),
                            "ESTACAO": first_8_lines[2].split(';')[1].strip(),
                            "CODIGO_WMO": first_8_lines[3].split(';')[1].strip(),
                            "LATITUDE": first_8_lines[4].split(';')[1].strip(),
                            "LONGITUDE": first_8_lines[5].split(';')[1].strip(),
                            "ALTITUDE": first_8_lines[6].split(';')[1].strip(),
                            "DATA_FUNDACAO": first_8_lines[7].split(';')[1].strip()
                        }

                        # Lê o resto do arquivo ignorando as 8 primeiras linhas
                        temp_df = pd.read_csv(
                            file_path,
                            skiprows=8,
                            encoding='latin1',
                            sep=';',  # Especifica o delimitador correto
                            error_bad_lines=False,  # Ignora linhas com erro
                            warn_bad_lines=True,    # Mostra as linhas problemáticas
                            engine='python'         # Usa o motor Python para maior flexibilidade
                        )
                        # Adiciona as variáveis extraídas como novas colunas
                        for key, value in metadata.items():
                            temp_df[key] = value

                        # Adiciona o arquivo lido ao DataFrame concatenado
                        concatenated_df = pd.concat([concatenated_df, temp_df], ignore_index=True)
                    except Exception as e:
                        print(f"Erro ao processar {file_path}: {e}")
                else:
                    print(f"Arquivo já processado ou não encontrado: {file_path}")
        else:
            print(f"Ano não encontrado no arquivo {base_path}")

    # Remove duplicatas do DataFrame final considerando todas as colunas
    concatenated_df = concatenated_df.drop_duplicates()

    return concatenated_df
        
# retira warnings
import warnings
warnings.filterwarnings('ignore')

# Carrega os dados de acidentes com estações climáticas
datas_climaticas = load_and_concat_unique_data_with_metadata(acidentes_com_estacoes)
datas_climaticas.head()

Lendo arquivo: ../data/INMET/2020/INMET_S_SC_A806_FLORIANOPOLIS_01-01-2020_A_31-12-2020.CSV
Lendo arquivo: ../data/INMET/2021/INMET_S_SC_A806_FLORIANOPOLIS_01-01-2021_A_31-12-2021.CSV
Lendo arquivo: ../data/INMET/2022/INMET_S_SC_A806_FLORIANOPOLIS_01-01-2022_A_31-12-2022.CSV
Lendo arquivo: ../data/INMET/2023/INMET_S_SC_A806_FLORIANOPOLIS_01-01-2023_A_31-12-2023.CSV
Lendo arquivo: ../data/INMET/2020/INMET_S_SC_A868_ITAJAI_01-01-2020_A_31-12-2020.CSV
Lendo arquivo: ../data/INMET/2021/INMET_S_SC_A868_ITAJAI_01-01-2021_A_31-12-2021.CSV
Lendo arquivo: ../data/INMET/2022/INMET_S_SC_A868_ITAJAI_01-01-2022_A_31-12-2022.CSV
Lendo arquivo: ../data/INMET/2023/INMET_S_SC_A868_ITAJAI_01-01-2023_A_31-12-2023.CSV
Lendo arquivo: ../data/INMET/2020/INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2020_A_31-12-2020.CSV
Lendo arquivo: ../data/INMET/2021/INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2021_A_31-12-2021.CSV
Lendo arquivo: ../data/INMET/2022/INMET_S_SC_A866_Laguna  - Farol de Santa

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,"VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,REGIAO,UF,ESTACAO,CODIGO_WMO,LATITUDE,LONGITUDE,ALTITUDE,DATA_FUNDACAO
0,2020/01/01,0000 UTC,0,"1008,5","1008,7","1007,7",NaN,"26,3","21,6","26,6",...,"3,1",NaN,S,SC,FLORIANOPOLIS,A806,"-27,60253","-48,620096","4,87",22/01/03
1,2020/01/01,0100 UTC,0,"1009,3","1009,5","1008,5",NaN,"26,5","21,5","26,5",...,2,NaN,S,SC,FLORIANOPOLIS,A806,"-27,60253","-48,620096","4,87",22/01/03
2,2020/01/01,0200 UTC,0,"1008,6","1009,3","1008,5",NaN,"26,2","21,4","26,5",...,"3,1",NaN,S,SC,FLORIANOPOLIS,A806,"-27,60253","-48,620096","4,87",22/01/03
3,2020/01/01,0300 UTC,0,"1007,9","1008,6","1007,9",NaN,"26,2","21,6","26,3",...,"2,3",NaN,S,SC,FLORIANOPOLIS,A806,"-27,60253","-48,620096","4,87",22/01/03
4,2020/01/01,0400 UTC,0,"1007,4","1007,9","1007,4",NaN,"25,4","21,7","26,2",...,"3,4",NaN,S,SC,FLORIANOPOLIS,A806,"-27,60253","-48,620096","4,87",22/01/03


In [74]:
# verifica duplicatas
print(datas_climaticas.duplicated().sum())
# verifica Data value counts
print(datas_climaticas['Data'].value_counts())

0
2020/01/01    96
2022/09/09    96
2022/09/07    96
2022/09/06    96
2022/09/05    96
              ..
2021/04/30    96
2021/04/29    96
2021/04/28    96
2021/04/27    96
2023/12/31    96
Name: Data, Length: 1461, dtype: int64


In [75]:
datas_climaticas.to_csv('datas_climaticas.csv', index=False) 

In [76]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime, timedelta

def find_nearest_weather_station_with_time(accidents_df, weather_stations_df, radius_km=50):
    # Função para converter vírgulas para ponto e depois para float
    def convert_to_float(value):
        if isinstance(value, str):  # Verifica se o valor é uma string
            value = value.replace(',', '.')  # Substitui vírgula por ponto
        return float(value) if value else 0.0  # Converte para float, ou 0.0 se for None ou vazio

    # Copiar os DataFrames de entrada para evitar modificar os originais
    accidents_df = accidents_df.copy()
    weather_stations_df = weather_stations_df.copy()

    # Remover linhas com valores NaN em coordenadas
    accidents_df = accidents_df.dropna(subset=['latitude', 'longitude'])
    weather_stations_df = weather_stations_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

    # Converter as colunas de latitude e longitude para float (com correção de vírgula)
    weather_stations_df['LATITUDE'] = weather_stations_df['LATITUDE'].apply(convert_to_float)
    weather_stations_df['LONGITUDE'] = weather_stations_df['LONGITUDE'].apply(convert_to_float)

    # Converter as colunas de data e hora para datetime
    accidents_df['data_hora'] = pd.to_datetime(accidents_df['data_hora'])
    
    # Ajuste da coluna de hora no dataframe de clima
    weather_stations_df['datetime'] = pd.to_datetime(weather_stations_df['Data'] + ' ' + weather_stations_df['Hora UTC'].str.replace(' UTC', ''), format='%Y/%m/%d %H%M')

    # Lista para armazenar resultados
    merged_data = []

    # Itera sobre cada linha do DataFrame de acidentes
    for _, accident_row in accidents_df.iterrows():
        accident_location = (accident_row['latitude'], accident_row['longitude'])
        accident_time = accident_row['data_hora']

        print(f"Procurando estação climática mais próxima para o acidente em {accident_location} na hora {accident_time}")

        # Calcula a distância para cada estação climática
        weather_stations_df['distance'] = weather_stations_df.apply(
            lambda row: geodesic(accident_location, (row['LATITUDE'], row['LONGITUDE'])).kilometers, axis=1
        )

        # Filtra estações dentro do raio especificado
        nearby_stations = weather_stations_df[weather_stations_df['distance'] <= radius_km]

        if not nearby_stations.empty:
            # Encontrar a estação mais próxima em termos de data/hora
            nearby_stations['time_diff'] = (nearby_stations['datetime'] - accident_time).abs()

            # Encontra a estação com a menor diferença de tempo
            nearest_station = nearby_stations.loc[nearby_stations['time_diff'].idxmin()]
            
            # Combina os dados do acidente e da estação climática
            combined_row = {**accident_row.to_dict(), **nearest_station.to_dict()}
        else:
            # Se nenhuma estação estiver dentro do raio, preenche com NaN
            combined_row = {**accident_row.to_dict(), **{col: np.nan for col in weather_stations_df.columns}}

        merged_data.append(combined_row)

    # Retorna o DataFrame combinado
    return pd.DataFrame(merged_data)


acidentes_com_estacoes = find_nearest_weather_station_with_time(acidentes_sample, datas_climaticas)
acidentes_com_estacoes


Procurando estação climática mais próxima para o acidente em (-27.975972, -48.679295) na hora 2021-09-29 01:20:00
Procurando estação climática mais próxima para o acidente em (-26.60108, -48.72536) na hora 2021-03-30 07:31:00
Procurando estação climática mais próxima para o acidente em (-27.70439, -48.6586) na hora 2024-02-24 03:05:00
Procurando estação climática mais próxima para o acidente em (-26.85054, -48.72587) na hora 2020-05-22 03:02:00
Procurando estação climática mais próxima para o acidente em (-28.445025, -48.967059) na hora 2021-11-06 12:41:00
Procurando estação climática mais próxima para o acidente em (-26.85488, -48.72722) na hora 2023-11-28 11:06:00
Procurando estação climática mais próxima para o acidente em (-27.00178, -48.6376) na hora 2021-11-01 06:49:00
Procurando estação climática mais próxima para o acidente em (-27.47249, -48.64114) na hora 2020-09-08 08:34:00
Procurando estação climática mais próxima para o acidente em (-26.28057, -48.89053) na hora 2022-08-13

,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,UF,ESTACAO,CODIGO_WMO,LATITUDE,LONGITUDE,ALTITUDE,DATA_FUNDACAO,datetime,distance,time_diff
0,80,254.0,BR-101/SC,Decrescente,Tombamento,0,0,1,0,0,...,SC,FLORIANOPOLIS,A806,-27.602530,-48.620096,"4,87",22/01/03,2021-09-29 01:00:00,41.792599,0 days 00:20:00
1,56,80.5,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,SC,ITAJAI,A868,-26.950833,-48.761944,"9,76",24/06/10,2021-03-30 08:00:00,38.923092,0 days 00:29:00
2,274,221.3,BR-101/SC,Decrescente,Colisão Lateral,0,0,1,1,0,...,SC,FLORIANOPOLIS,A806,-27.602500,-48.620000,"4,87",22/01/03,2023-12-31 23:00:00,11.915959,54 days 04:05:00
3,280,112.7,BR-101/SC,Decrescente,Colisão Lateral,1,0,1,0,0,...,SC,ITAJAI,A868,-26.950924,-48.762031,"9,76",24/06/10,2020-05-22 03:00:00,11.688459,0 days 00:02:00
4,72,328.0,BR-101/SC,Crescente,Colisão - Traseira,1,0,0,1,0,...,SC,Laguna - Farol de Santa Marta,A866,-28.604444,-48.813333,"34,36",01/06/08,2021-11-06 13:00:00,23.207126,0 days 00:19:00
5,390,113.2,BR-101/SC,Decrescente,Colisão Traseira,1,0,1,0,0,...,SC,ITAJAI,A868,-26.950833,-48.761944,"9,76",24/06/10,2023-11-28 11:00:00,11.177425,0 days 00:06:00
6,378,133.9,BR-101/SC,Crescente,Colisão Traseira,1,0,0,1,0,...,SC,ITAJAI,A868,-26.950833,-48.761944,"9,76",24/06/10,2021-11-01 07:00:00,13.573882,0 days 00:11:00
7,147,190.5,BR-101/SC,Decrescente,"Choque - Defensa, barreira ou submarino""""",1,0,0,0,0,...,SC,FLORIANOPOLIS,A806,-27.602530,-48.620096,"4,87",22/01/03,2020-09-08 09:00:00,14.559155,0 days 00:26:00
8,71,36.5,BR-101/SC,Decrescente,Capotamento,1,0,0,0,0,...,SC,ITAPOA,A851,-26.081389,-48.641667,"6,18",06/06/07,2022-08-13 06:00:00,33.254348,0 days 00:29:00
9,207,80.5,BR-101/SC,Crescente,Colisão Traseira,1,0,1,0,0,...,SC,ITAJAI,A868,-26.950833,-48.761944,"9,76",24/06/10,2023-03-28 12:00:00,38.907927,0 days 00:23:00


In [ ]:
acidentes_com_estacoes.to_csv('acidentes_com_estacoes.csv', index=False)

In [ ]:
# Tente com 'latin1' ou 'iso-8859-1'
# clima_df = pd.read_csv('../data/INMET/2023/INMET_SE_SP_A768_TUPA_01-01-2023_A_31-12-2023.csv', 
#                        sep=';', 
#                        skiprows=8, 
#                        encoding='latin1')  # Ou experimente 'iso-8859-1'
# clima_df.head(2)

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2023/01/01,0000 UTC,0,957,957,"955,8",NaN,"23,1","23,1","23,8",23,"23,6","22,9",100.0,95.0,100.0,101.0,"3,5","2,1",NaN
1,2023/01/01,0100 UTC,0,"957,2","957,2","956,8",NaN,"23,3","23,3","23,6","23,2","23,4","22,8",100.0,95.0,100.0,82.0,"3,5",2,NaN


In [ ]:
# acidentes_df.columns

Index(['n_da_ocorrencia', 'km', 'trecho', 'sentido', 'tipo_de_acidente',
       'automovel', 'bicicleta', 'caminhao', 'moto', 'onibus', 'outros',
       'tracao_animal', 'transporte_de_cargas_especiais', 'trator_maquinas',
       'utilitarios', 'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos', 'concessionaria', 'data_hora',
       'com_vitima', 'estado', 'ano', 'mes', 'latitude', 'longitude',
       'geometry'],
      dtype='object')

In [ ]:
# df = pd.read_csv('rodo_estacoes.csv')
# df

,Unnamed: 0,rodovia,sentido,concessionaria,km,ano_do_pnv_snv,geometry,estacoes_proximas
0,0,092/PR,Decrescente,LITORAL PIONEIRO,324.090,2019,"LINESTRING (-49.732399 -24.251814, -49.734715 ...",499 JOAQUIM TAVORA\n1087 JOAQUIM TAVORA...
1,1,151/PR,Crescente,LITORAL PIONEIRO,318.022,2019,"LINESTRING (-49.479561 -24.116515, -49.485662 ...",497 CASTRO\n1085 CASTRO\n2218 CASTRO...
2,2,239/PR,Decrescente,LITORAL PIONEIRO,12.359,2019,"LINESTRING (-49.377773 -24.118914, -49.386575 ...",NaN
3,3,407/PR,Crescente,LITORAL PIONEIRO,17.990,2019,"LINESTRING (-48.577489 -25.568841, -48.572851 ...",NaN
4,4,408/PR,Crescente,LITORAL PIONEIRO,21.999,2019,"LINESTRING (-48.75282 -25.525566, -48.762374 -...",514 MORRETES\n1102 MORRETES\n1668 MO...
...,...,...,...,...,...,...,...,...
94,94,CONTORNO SUL - FS1/BA,Decrescente,VIA BAHIA,7.900,2003,"LINESTRING (-38.988861 -12.268139, -38.988861 ...",130 FEIRA DE SANTANA\n718 FEIRA DE SAN...
95,95,CW-116/PR,Decrescente,AUTOPISTA REGIS BITTENCOURT,18.500,2017,"LINESTRING (-49.044907 -25.345556, -49.044907 ...",495 CURITIBA\n520 COLOMBO\n1083 CU...
96,96,Contorno de Iconha/ES,Decrescente,ECO101,7.044,2020,"LINESTRING (-40.783414 -20.78697, -40.783414 -...",NaN
97,97,Contorno de Uberlândia/MG,Decrescente,ECO050,21.000,2017,"LINESTRING (-48.256849 -18.859859, -48.256917 ...",362 UBERLANDIA\n950 UBERLANDIA\n1522 ...
